In [ ]:
import os
from os import path

# Third-party
from astropy.constants import G
from astropy.io import fits, ascii
from astropy.stats import median_absolute_deviation
from astropy.table import Table, QTable, join
from astropy.time import Time
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import h5py
import pandas as pd
from sqlalchemy import func
import tqdm
from scipy.stats import beta

from thejoker import JokerSamples

from twoface.config import TWOFACE_CACHE_PATH
from twoface.samples_analysis import MAP_sample
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, NessRG,
                        StarResult, Status, JokerRun)
from twoface.plot import plot_two_panel, plot_phase_fold
from twoface.mass import get_m2_min, mf, period_at_surface
from twoface.log import log as logger
logger.setLevel(100)

In [ ]:
plot_path = '../../paper/1-catalog/figures/'
table_path = '../../paper/1-catalog/tables/'

In [ ]:
Session, _ = db_connect(path.join(TWOFACE_CACHE_PATH, 'apogee.sqlite'))
session = Session()

samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter.hdf5')
mcmc_samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter-mcmc.hdf5')

In [ ]:
unimodal = QTable.read(path.join(table_path, 'highK-unimodal.fits'))
bimodal = QTable.read(path.join(table_path, 'highK-bimodal.fits'))
clean_uni = unimodal[unimodal['clean_flag'] == 0]
clean_bi = bimodal[bimodal['clean_flag'] == 0]

print('{0} unimodal, {1} clean'.format(len(unimodal), len(clean_uni)))
print('{0} bimodal, {1} clean'.format(len(bimodal)//2, len(clean_bi)//2))

## Which were already in Troup?

In [ ]:
troup = ascii.read('../../../papers/thejoker-paper/data/troup16-dr12.csv', format='commented_header')

In [ ]:
n_uni = len(clean_uni['APOGEE_ID'])
n_uni_troup = np.isin(clean_uni['APOGEE_ID'], troup['APOGEE_ID']).sum()

In [ ]:
n_bi = len(np.unique(clean_bi['APOGEE_ID']))
n_bi_troup = np.isin(np.unique(clean_bi['APOGEE_ID']), troup['APOGEE_ID']).sum()

In [ ]:
assert np.isin(unimodal['APOGEE_ID'], bimodal['APOGEE_ID']).sum() == 0

In [ ]:
print('{0} new unimodal companions'.format(n_uni - n_uni_troup))
print('{0} new bimodal companions'.format(n_bi - n_bi_troup))

## Masses

In [ ]:
def get_asini(P, e, K, m1, m2):
    return (P/(2*np.pi) * np.sqrt(1-e**2) * K * (m1+m2) / m2).to(u.au)

def get_a2sini(P, e, K, m1, m2):
    return (P/(2*np.pi) * (m1/m2) * np.sqrt(1-e**2) * K).to(u.au)

In [ ]:
uni_cat = clean_uni[(clean_uni['LOGG'] > 2) & np.isfinite(clean_uni['M1'])]

bi_logg = np.array([AllStar.get_apogee_id(session, aid).logg 
                    for aid in clean_bi['APOGEE_ID']])
bi_cat = clean_bi[(bi_logg > 2) & np.isfinite(clean_bi['M1']) & np.isfinite(clean_bi['M2_min'])]
print(len(uni_cat), len(bi_cat)//2)

fig, axes = plt.subplots(1, 2, figsize=(11, 4.8))

ax = axes[0]

uni_cats = [uni_cat[uni_cat['converged']], uni_cat[~uni_cat['converged']]]
uni_colors = ['k', '#aaaaaa']
uni_zorders = [10, -10]
for cat, color, zorder in zip(uni_cats, uni_colors, uni_zorders):
    ax.errorbar(cat['M1'].value, cat['M2_min'].value,
                xerr=cat['M1_err'].value,
                yerr=cat['M2_min_err'].value,
                marker='o', linestyle='none', 
                markersize=5, color=color, alpha=0.8, zorder=zorder)

ax.scatter(bi_cat['M1'].value, bi_cat['M2_min'].value,
           marker='s', color='tab:blue', alpha=0.8, linewidth=0, s=13)
ax.plot(np.vstack((bi_cat['M1'].value[::2], bi_cat['M1'].value[1::2])), 
        np.vstack((bi_cat['M2_min'].value[::2], bi_cat['M2_min'].value[1::2])), 
        marker='', linestyle='-', color='tab:blue', alpha=0.4)

ax.plot(np.logspace(-3, 1, 1024), 
        np.logspace(-3, 1, 1024),
        marker='', color='#aaaaaa', zorder=-100, linestyle='--')
ax.axhline(0.08, marker='', color='#aaaaaa', zorder=-100, linestyle='--')

# ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(0.5, 3)
ax.xaxis.set_ticks(np.arange(0.5, 3+0.1, 0.5))
ax.set_ylim(0.01, 10)
ax.set_ylabel(r'$M_{2, {\rm min}}$ ' + '[{0:latex_inline}]'.format(u.Msun))

axr = ax.twinx()
ylim = ax.get_ylim()
axr.set_yscale('log')
axr.set_ylim((ylim[0]*u.Msun).to(u.Mjup).value, 
             (ylim[1]*u.Msun).to(u.Mjup).value)
axr.set_ylabel('[{0:latex_inline}]'.format(u.Mjup))

ax.set_xlabel('$M_1$ ' + '[{0:latex_inline}]'.format(u.Msun))

ax.text(2.95, 3, r'$M_{2, {\rm min}} = M_1$', 
        ha='right', va='bottom', fontsize=15, color='#555555')
ax.text(2.95, 0.07, r'$M_{2, {\rm min}} = 0.08\,{\rm M}_\odot$', 
        ha='right', va='top', fontsize=15, color='#555555')

# -- next panel

ax = axes[1]

for cat, color, zorder in zip(uni_cats, uni_colors, uni_zorders):
    print(len(cat))
    
    q = cat['M2_min'] / cat['M1']
    asini = get_asini(cat['P'], cat['e'], cat['K'], cat['M1'], cat['M2_min'])
    R1 = np.sqrt(G * cat['M1'] / (10**cat['LOGG']*u.cm/u.s**2)).to(u.au)
    
    xerr = q * np.sqrt((cat['M2_min_err']/cat['M2_min'])**2 + (cat['M1_err']/cat['M1'])**2).value    
    ns = 10000
    asinis = get_asini(np.random.normal(cat['P'], cat['P_err'], (ns, len(cat)))*cat['P'].unit, 
                       np.random.normal(cat['e'], cat['e_err'], (ns, len(cat))),
                       np.random.normal(cat['K'], cat['K_err'], (ns, len(cat)))*cat['K'].unit,
                       np.random.normal(cat['M1'], cat['M1_err'], (ns, len(cat)))*cat['M1'].unit,
                       np.random.normal(cat['M2_min'], cat['M2_min_err'], (ns, len(cat)))*cat['M2_min'].unit)

    loggs = np.random.normal(cat['LOGG'], cat['LOGG_ERR'], (ns, len(cat)))
    R1s = np.sqrt(G * np.random.normal(cat['M1'], cat['M1_err'], (ns, len(cat)))*cat['M1'].unit / (10**loggs*u.cm/u.s**2)).to(u.au)
    yerr = np.std(R1s/asinis, axis=0)
    
    ax.errorbar(q, R1/asini,
                xerr=xerr, yerr=yerr,
                marker='o', linestyle='none', zorder=zorder,
                markersize=5, color=color, alpha=0.8)

bi_logg = np.array([AllStar.get_apogee_id(session, aid).logg 
                    for aid in clean_bi['APOGEE_ID']])
bi_cat = clean_bi[bi_logg > 2]
bi_logg = bi_logg[bi_logg > 2]
q = bi_cat['M2_min'] / bi_cat['M1']
asini = get_asini(bi_cat['P'], bi_cat['e'], bi_cat['K'], bi_cat['M1'], bi_cat['M2_min'])
R1 = np.sqrt(G * bi_cat['M1'] / (10**bi_logg*u.cm/u.s**2)).to(u.au)
_y = R1/asini
ax.scatter(q, _y, 
           marker='s', color='tab:blue', alpha=0.8, linewidth=0, s=13)
ax.plot(np.vstack((q[::2], q[1::2])), 
        np.vstack((_y[::2], _y[1::2])), 
        marker='', linestyle='-', color='tab:blue', alpha=0.4)

qgrid = np.linspace(1E-2, 1E1)
r1 = 0.49*qgrid**(-2/3.) / (0.6*qgrid**(-2/3.) + np.log(1+qgrid**(-1/3)))

ax.plot(qgrid, r1, marker='', color='#aaaaaa', zorder=-100, linestyle='--')
ax.set_xlim(1e-2, 1E1)
ax.set_ylim(1e-3, 1E0)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'$q_{\rm min}$')
ax.set_ylabel(r'$R_1 / (a\,\sin i)$')

fig.tight_layout()
fig.subplots_adjust(wspace=0.45)
fig.savefig(path.join(plot_path, 'mass.pdf'))

In [ ]:
# Both samples at q>1
bi_cat[::2][((bi_cat['M2_min'].value[::2] / bi_cat['M1'].value[::2]) > 1) & 
            ((bi_cat['M2_min'].value[1::2] / bi_cat['M1'].value[1::2]) > 1)]

In [ ]:
# Both samples at M2<0.08 Msun
bi_cat[::2][((bi_cat['M2_min'][::2]) < 0.08*u.Msun) & 
            ((bi_cat['M2_min'][1::2]) < 0.08*u.Msun)]

In [ ]:
bi_cat[1::2][((bi_cat['M2_min'][::2]) < 0.08*u.Msun) & 
            ((bi_cat['M2_min'][1::2]) < 0.08*u.Msun)]

### The interesting 3:

In [ ]:
from thejoker.data import RVData

In [ ]:
_plot_path = '../../plots/three-interesting'
os.makedirs(_plot_path, exist_ok=True)

with h5py.File(samples_file) as f:
    for apogee_id in ['2M01231070+1801407', # BH
                      '2M04582049+2232000', # BD
                      '2M20130687+2044173']: # BD
        samples = JokerSamples.from_hdf5(f[apogee_id])
        star = AllStar.get_apogee_id(session, apogee_id)
        data = star.apogeervdata()
        
        rows = bi_cat[bi_cat['APOGEE_ID'] == apogee_id]
        a2s = [get_a2sini(r['P'], r['e'], r['K'], r['M1'], r['M2_min']) 
               for r in rows]
        print('in Troup: {0}\n'.format(apogee_id in troup['APOGEE_ID']) + 
              'logg = {0:.2f}, Teff = {1:.1f}\n'.format(star.logg, star.teff) + 
              'a2sini = {0:.3f} or {1:.3f}'.format(*a2s))
        
        # with LAMOST RV: http://dr3.lamost.org/spectrum/view?obsid=268505101
        data = RVData(t=Time(np.append(data.t.mjd, [56980.]), format='mjd'),
                      rv=np.append(data.rv.value, 11.1)*u.km/u.s,
                      stddev=np.append(data.stddev.value, 1.)*u.km/u.s)
        
        fig = plot_two_panel(data, samples)
        fig.axes[0].set_title(star.apogee_id)
        fig.tight_layout()
#         fig.savefig(path.join(_plot_path, '{0}.png'.format(apogee_id)), dpi=250)

        break

In [ ]:
star.k

In [ ]:
samples['P'].min(), samples['P'].max()

In [ ]:
with h5py.File('../../cache/2M01231070+1801407.hdf5') as f:
    new_samples = JokerSamples.from_hdf5(f)
    
# new_samples = samples

In [ ]:
fig = plot_two_panel(data, new_samples)
fig.axes[0].set_title('{0} - {1} samples'.format(star.apogee_id, len(new_samples)))
fig.tight_layout()

fig = plot_two_panel(data, samples)
fig.axes[0].set_title('{0} - {1} samples'.format(star.apogee_id, len(samples)))
fig.tight_layout()

In [ ]:
m1 = np.exp(star.ness_rg.lnM) * u.Msun

m2s = np.zeros(len(new_samples)) * u.Msun
for i in range(len(new_samples)):
    mf = new_samples.get_orbit(i).m_f
    m2s[i] = get_m2_min(m1, mf)
    
m2s_2 = np.zeros(len(samples)) * u.Msun
for i in range(len(samples)):
    mf = samples.get_orbit(i).m_f
    m2s_2[i] = get_m2_min(m1, mf)

In [ ]:
_, bins, _ = plt.hist(m2s.value, bins='auto', normed=True);
plt.hist(m2s_2.value, bins=bins, normed=True);
plt.axvline(m1.value)
plt.gca().xaxis.set_ticks(np.arange(1, 6, 0.5))

In [ ]:
# ptf = ascii.read('/Users/adrian/Downloads/irsa_catalog_search_results.tbl')
# ptf = ptf[(ptf['oid'] == 35322100000378) & (ptf['fid'] == 2)]

# plt.errorbar(ptf['obsmjd'], ptf['mag_autocorr'], 
#              yerr=ptf['magerr_auto'], linestyle='none', marker='o')
# plt.ylim(14.0, 13.8)

In [ ]:
wise = ascii.read('/Users/adrian/Downloads/irsa_catalog_search_results_wise.tbl')
wise = wise[(wise['source_id_mf'] == '0206p181_ac51-027396')]

In [ ]:
plt.errorbar(((wise['mjd']) / new_samples['P'].value) % 1, wise['w1mpro_ep'], wise['w1sigmpro_ep'],
             linestyle='none', marker='o')
plt.ylim(12.2, 11.7)

---

## Period-ecc-logg

In [ ]:
from scipy.stats import binned_statistic

In [ ]:
# bins = np.linspace(0, 3.1, 15)
# res15 = binned_statistic(np.log10(cat['P'].value), values=cat['e'], bins=bins,
#                          statistic=lambda x: np.percentile(x, 15))
# res50 = binned_statistic(np.log10(cat['P'].value), values=cat['e'], bins=bins,
#                          statistic=lambda x: np.percentile(x, 50))
# res85 = binned_statistic(np.log10(cat['P'].value), values=cat['e'], bins=bins,
#                          statistic=lambda x: np.percentile(x, 85))

# binc = (res15.bin_edges[:-1]+res15.bin_edges[1:])/2
# plt.plot(binc, res15.statistic)
# plt.plot(binc, res50.statistic)
# plt.plot(binc, res85.statistic)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 8.5), sharex=True, sharey=True)

grid = np.arange(2, 3.5+0.1, 0.3)
for i, (l, r) in enumerate(zip(grid, grid+0.5)):
    mask = (cat['LOGG'] > l) & (cat['LOGG'] <= r)
    subcat = cat[mask]
    axes.flat[i].errorbar(subcat['P'].value, subcat['e'], 
                          xerr=subcat['P_err'].value, yerr=subcat['e_err'],
                          marker='o', linestyle='none', alpha=0.8)
    axes.flat[i].axvline(10., zorder=-10, color='#aaaaaa', linestyle='--')
    axes.flat[i].axvline(100., zorder=-10, color='#aaaaaa', linestyle='--')
    axes.flat[i].text(1, 0.95, r'${0:.2f} < \log g \leq {1:.2f}$'.format(l, r), 
                      ha='left', va='top', fontsize=16, backgroundcolor='w')
    
    
axes.flat[0].set_xlim(0.8, 1800)
axes.flat[0].set_ylim(-0.05, 1.)
axes.flat[0].set_xscale('log')

axes[1, 1].set_xlabel('period, $P$ [day]')
axes[0, 0].set_ylabel('eccentricity, $e$')
axes[1, 0].set_ylabel('eccentricity, $e$')

fig.tight_layout()

fig.savefig(path.join(plot_path, 'P-e-grid.pdf'))

In [ ]:
cat = clean_uni[clean_uni['LOGG'] > 2]

fig, axes = plt.subplots(1, 2, figsize=(9, 5.5), sharex=True)

cs = axes[0].scatter(cat['P'], cat['LOGG'], 
                     c=cat['e'], alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=0, vmax=1, 
                     cmap='magma')

axes[0].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100)

axes[0].set_xlabel(r'$P$ [day]')
axes[0].set_ylabel(r'$\log g$')

axes[0].set_xlim(0.4, 3000)
axes[0].set_ylim(4, 0)
axes[0].set_xscale('log')

cax = fig.add_axes([0.11, 0.85, 0.402, 0.03])
cb = fig.colorbar(cs, cax=cax, orientation='horizontal')
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
cb.set_ticks(np.arange(0, 1+0.1, 0.2))
cb.set_label('$e$', labelpad=10)
cb.solids.set_rasterized(True) 
cb.solids.set_edgecolor('face')

# ---

cs = axes[1].scatter(cat['P'], cat['e'], 
                     c=cat['LOGG'], alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=1, vmax=4, 
                     cmap='viridis')

axes[1].scatter(cat['P'][cat['DR14RC']], cat['e'][cat['DR14RC']], 
                linewidth=1, edgecolor='tab:red', facecolor='none', s=40,
                marker='o', zorder=100)

axes[1].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100)

# axes[1].set_ylabel(r'$P / P_{\rm surface}$')
axes[1].set_ylabel(r'eccentricity, $e$')

axes[1].set_ylim(-0.05, 1)
# axes[1].set_yscale('log')
axes[1].yaxis.set_ticks(np.arange(0, 1+0.2, 0.2))

cax = fig.add_axes([0.565, 0.85, 0.402, 0.03])
cb = fig.colorbar(cs, cax=cax, orientation='horizontal')
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
cb.set_label(r'$\log g$', labelpad=10)
cb.solids.set_rasterized(True) 
cb.solids.set_edgecolor('face')

fig.tight_layout()
fig.subplots_adjust(top=0.8, wspace=0.125)

# fig.savefig(path.join(plot_path, 'P-logg-e.pdf'))

In [ ]:
cat = clean_uni[clean_uni['LOGG'] > 2]

fig, axes = plt.subplots(1, 2, figsize=(10.5, 5.), sharey=True)

P_surface = period_at_surface(1.36*u.Msun, cat['LOGG'], 
                              cat['e'], 0.5*u.Msun)

cs = axes[0].scatter(cat['e'], cat['P']/P_surface, 
                     c=cat['LOGG'], alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=2, vmax=4, 
                     cmap='viridis')

axes[0].scatter(cat['e'][cat['DR14RC']], (cat['P']/P_surface)[cat['DR14RC']], 
                linewidth=1, edgecolor='tab:red', facecolor='none', s=40,
                marker='o', zorder=100)

axes[0].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[0].set_ylabel(r'$P / P_{\rm surface}$')
axes[0].set_xlabel(r'eccentricity, $e$')

axes[0].set_xlim(-0.05, 1)
axes[0].set_ylim(0.4, 2000)
axes[1].set_yscale('log')
axes[0].xaxis.set_ticks(np.arange(0, 1+0.2, 0.2))

# ---

cs = axes[1].scatter(cat['K'], cat['P']/P_surface, 
                     c=cat['LOGG'], alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=2, vmax=4, 
                     cmap='viridis')

axes[1].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[1].set_xlim(1, 250)
axes[1].set_xscale('log')
axes[1].set_xlabel('RV amplitude, $K$ [{0:latex_inline}]'.format(u.km/u.s))

cax = fig.add_axes([0.87, 0.185, 0.02, 0.76])
cb = fig.colorbar(cs, cax=cax)
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
cb.set_label(r'$\log g$', labelpad=10)
cb.solids.set_rasterized(True) 
cb.solids.set_edgecolor('face')
cb.set_ticks(np.arange(2, 4+0.1, 0.5))

fig.tight_layout()
fig.subplots_adjust(right=0.85)

# fig.savefig(path.join(plot_path, 'P-e-K.pdf'))

In [ ]:
cat = clean_uni[clean_uni['LOGG'] > 2]

fig, ax = plt.subplots(1, 1, figsize=(6, 5))

bins = np.linspace(0, 1, 15)
bin_c = (bins[:-1]+bins[1:]) / 2
# cut = cat['P'] < 10*u.day

P_surface = period_at_surface(1.36*u.Msun, cat['LOGG'], 
                              cat['e'], 0.5*u.Msun)
cut = (cat['P']/P_surface) < 8

# ax.hist(cat['e'][cut], bins=bins, 
#         alpha=0.5, normed=True, rasterized=True,
#         label=r'$P < 20\,{\rm d}$' + '  $(N = {0})$'.format(cut.sum()));
# ax.hist(cat['e'][~cut], bins=bins, 
#         alpha=0.5, normed=True, rasterized=True,
#         label=r'$P \geq 20\,{\rm d}$' + '  $(N={0})$'.format(np.logical_not(cut).sum()));

n, *_ = np.histogram(cat['e'][cut], bins=bins, normed=True)
ax.plot(bin_c, n, 
        alpha=0.9, drawstyle='steps-mid', marker='', lw=2.,
        label=r'$P < 10\,{\rm d}$' + '  $(N = {0})$'.format(cut.sum()));

n, *_ = np.histogram(cat['e'][~cut], bins=bins, normed=True)
ax.plot(bin_c, n, 
        alpha=0.9, drawstyle='steps-mid', marker='', lw=2.,
        label=r'$P \geq 10\,{\rm d}$' + '  $(N={0})$'.format(np.logical_not(cut).sum()));

egrid = np.linspace(0, 1, 256)
ax.plot(egrid, beta.pdf(egrid, 0.867, 3.03),
        marker='', linestyle='--', color='k', alpha=0.4,
        label='prior', zorder=-10)

ax.legend(loc='best', fontsize=16)
ax.set_xlabel('eccentricity, $e$')
ax.set_ylabel('$p(e)$')

ax.set_xlim(-0.02, 1.02)
ax.set_yscale('log')
ax.set_ylim(8E-2, 2E1)

fig.tight_layout()

# fig.savefig(path.join(plot_path, 'ecc-dist.pdf'), dpi=250)

In [ ]:
plt.hist(cat['FE_H'], bins=np.linspace(-2.5, 0.75, 18));

Old idea: compute M2_min assuming M1 = 1.5

In [ ]:
cat = clean_cat.copy()
print(np.isfinite(cat['M2_min']).sum())

rnd = np.random.RandomState(seed=42)
N = rnd.normal

n_samples = 1024
for i, row in tqdm.tqdm(enumerate(cat)):
    if np.isfinite(row['M2_min']):
        continue
        
    m1_samples = N(1.5, 0.4, size=n_samples) * u.Msun
    mass_func = mf(P=N(row['P'].value, row['P_err'].value, n_samples) * row['P'].unit, 
                   K=N(row['K'].value, row['K_err'].value, n_samples) * row['K'].unit,
                   e=N(row['e'], row['e_err'], n_samples))
    m2_mins = get_m2_min(m1_samples, mass_func)
    
    row['M1'] = np.median(m1_samples)
    row['M1_err'] = 1.5 * median_absolute_deviation(m1_samples)
    
    row['M2_min'] = np.nanmedian(m2_mins)
    row['M2_min_err'] = 1.5 * median_absolute_deviation(m2_mins, ignore_nan=True)